In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import requests
from requests import get
import sqlite3


In [4]:
pages = [
    '?page=1','?page=2','?page=3','?page=4','?page=5','?page=6','?page=7','?page=8','?page=9','?page=10',
    '?page=11','?page=12','?page=13','?page=14'
]

a_info = []
album_info = []
genre_info = []
review_links = []
pdates = []

for page in pages:
    
    page = requests.get("https://pitchfork.com/reviews/albums/" + str(page))
   
    
    #parse
    soup = bs(page.text, 'html.parser')
                       
    artist = soup.find_all('div',class_='review')

   

    for g in artist:
    
        # ARTIST NAME INFO
        artist_info = g.find('ul', class_ = 'artist-list review__title-artist')
        artist_info2 = artist_info.li.text
        a_info.append(artist_info2)

        # ALBUM NAME
        album_name = g.find('h2',class_='review__title-album').text
        album_info.append(album_name)


        # GENRE
        gtypes1 = g.find('div',class_='review__meta')
        gtypes2 = gtypes1.find('li', class_='genre-list__item')
        genre_type = gtypes2.find('a',class_='genre-list__link').text
        genre_info.append(genre_type)


        # REVIEW URL
        core_url = 'https://pitchfork.com/'
        r_url = g.a['href']
        review_url = core_url + r_url
        review_links.append(review_url)


        # PUB_DATE
        pdate1 = g.find('div',class_='review__meta')
        pdate2 = pdate1.find('time', class_='pub-date')['datetime']
        pdates.append(pdate2)
        
        # pandas dataframe        
        genre_output = pd.DataFrame({
            'artist': a_info,
            'album': album_info,
            'genre': genre_info,
            'review_url': review_links,
            'pub_date': pdates,
            })


  
    




In [5]:
genre_output

,artist,album,genre,review_url,pub_date
0,The Replacements,Pleased to Meet Me (Deluxe Edition),Rock,https://pitchfork.com//reviews/albums/the-repl...,2020-10-15T05:00:00
1,Annie,Dark Hearts,Electronic,https://pitchfork.com//reviews/albums/annie-da...,2020-10-15T05:00:00
2,SPAZA,UPRIZE!,Jazz,https://pitchfork.com//reviews/albums/spaza-up...,2020-10-15T05:00:00
3,Gulfer,Gulfer,Rock,https://pitchfork.com//reviews/albums/gulfer-g...,2020-10-15T05:00:00
4,Future Islands,As Long As You Are,Rock,https://pitchfork.com//reviews/albums/future-i...,2020-10-14T05:00:00
...,...,...,...,...,...
163,The Front Bottoms,In Sickness & In Flames,Rock,https://pitchfork.com//reviews/albums/the-fron...,2020-08-26T05:00:00
164,Mary Chapin Carpenter,The Dirt and the Stars,Folk/Country,https://pitchfork.com//reviews/albums/mary-cha...,2020-08-26T05:00:00
165,Nubya Garcia,Source,Jazz,https://pitchfork.com//reviews/albums/nubya-ga...,2020-08-25T05:00:00
166,Cut Copy,"Freeze, Melt",Rock,https://pitchfork.com//reviews/albums/cut-copy...,2020-08-25T05:00:00


In [6]:
from pymongo import MongoClient

In [7]:
client = MongoClient()
col = client['musicgenre_DB']['rdata']

In [9]:
data_r = genre_output.to_dict(orient='records')
col.insert_many(data_r)